In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
# convert JSON data to pandas df
df = pd.read_json('crime-data.json')

df

,jurisdiction,year,population,murder,rape,robbery,agg_assault,b_e,larceny_theft,m_v_theft,...,overall_percent_change_per_100_000_people,violent_crime_rate_percent_change_per_100_000_people,property_crime_rate_percent_change_per_100_000_people,murder_rate_percent_change_per_100_000_people,rape_rate_percent_change_per_100_000_people,robbery_rate_percent_change_per_100_000_people,agg_assault_rate_percent_change_per_100_000_people,b_e_rate_percent_change_per_100_000_people,larceny_theft_rate_percent_change_per_100_000_people,m_v_theft_rate_percent_change_per_100_000_people
0,Allegany County,1975,79655,3,5,20,114,669,1425,93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allegany County,1976,83923,2,2,24,59,581,1384,73,...,-13.4,-41.8,-11.6,-36.7,-62.0,13.9,-50.9,-17.6,-7.8,-25.5
2,Allegany County,1977,82102,3,7,32,85,592,1390,102,...,6.4,49.2,4.5,53.3,257.8,36.3,47.3,4.2,2.7,42.8
3,Allegany County,1978,79966,1,2,18,81,539,1390,100,...,-1.0,-17.5,0.0,-65.8,-70.7,-42.2,-2.2,-6.5,2.7,0.7
4,Allegany County,1979,79721,1,7,18,84,502,1611,99,...,9.3,8.2,9.4,0.3,251.1,0.3,4.0,-6.6,16.3,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Washington County,2004,138016,5,17,88,426,699,2138,270,...,1.1,7.2,0.1,390.9,-27.4,-19.3,16.2,0.5,0.4,-2.9
996,Washington County,2005,140687,4,18,111,388,702,2306,261,...,2.1,-4.6,3.2,-21.5,3.9,23.7,-10.6,-1.5,5.8,-5.2
997,Washington County,2006,142284,4,28,158,378,786,2367,300,...,4.9,7.8,4.4,-1.1,53.8,40.7,-3.7,10.7,1.5,13.7
998,Washington County,2007,144219,7,38,137,324,679,2338,240,...,-7.7,-12.1,-6.9,72.7,33.9,-14.5,-15.4,-14.8,-2.6,-21.1


In [32]:
# cleaning data
df = df.fillna(0)
df = df.set_index(['jurisdiction', 'year'])

df

population  murder  rape  robbery  agg_assault  b_e  \
jurisdiction      year                                                        
Allegany County   1975       79655       3     5       20          114  669   
                  1976       83923       2     2       24           59  581   
                  1977       82102       3     7       32           85  592   
                  1978       79966       1     2       18           81  539   
                  1979       79721       1     7       18           84  502   
...                            ...     ...   ...      ...          ...  ...   
Washington County 2004      138016       5    17       88          426  699   
                  2005      140687       4    18      111          388  702   
                  2006      142284       4    28      158          378  786   
                  2007      144219       7    38      137          324  679   
                  2008      145781       3    15      116          308  701   

                        larceny_theft  m_v_theft  grand_total  \
jurisdiction      year                                          
Allegany County   1975           1425         93         2329   
                  1976           1384         73         2125   
                  1977           1390        102         2211   
                  1978           1390        100         2131   
                  1979           1611         99         2322   
...                               ...        ...          ...   
Washington County 2004           2138        270         3643   
                  2005           2306        261         3790   
                  2006           2367        300         4021   
                  2007           2338        240         3763   
                  2008           2234        246         3623   

                        violent_crime_total  ...  \
jurisdiction      year                       ...   
Allegany County   1975                  142  ...   
                  1976                   87  ...   
                  1977                  127  ...   
                  1978                  102  ...   
                  1979                  110  ...   
...                                     ...  ...   
Washington County 2004                  536  ...   
                  2005                  521  ...   
                  2006                  568  ...   
                  2007                  506  ...   
                  2008                  442  ...   

                        overall_percent_change_per_100_000_people  \
jurisdiction      year                                              
Allegany County   1975                                        0.0   
                  1976                                      -13.4   
                  1977                                        6.4   
                  1978                                       -1.0   
                  1979                                        9.3   
...                                                           ...   
Washington County 2004                                        1.1   
                  2005                                        2.1   
                  2006                                        4.9   
                  2007                                       -7.7   
                  2008                                       -4.8   

                        violent_crime_rate_percent_change_per_100_000_people  \
jurisdiction      year                                                         
Allegany County   1975                                                0.0      
                  1976                                              -41.8      
                  1977                                               49.2      
                  1978                                              -17.5      
                  1979                                                8.2      
...            

In [33]:
# normalize number of crimes commited for each county, for every year
df_crimes = df[['murder', 'rape', 'robbery', 'agg_assault', 'b_e', 'larceny_theft', 'm_v_theft']]
df_crimes_normed = df_crimes.divide(df_crimes.sum(axis=1), axis=0)

df_crimes_normed

murder      rape   robbery  agg_assault       b_e  \
jurisdiction      year                                                        
Allegany County   1975  0.001288  0.002147  0.008587     0.048948  0.287248   
                  1976  0.000941  0.000941  0.011294     0.027765  0.273412   
                  1977  0.001357  0.003166  0.014473     0.038444  0.267752   
                  1978  0.000469  0.000939  0.008447     0.038010  0.252933   
                  1979  0.000431  0.003015  0.007752     0.036176  0.216193   
...                          ...       ...       ...          ...       ...   
Washington County 2004  0.001372  0.004666  0.024156     0.116937  0.191875   
                  2005  0.001055  0.004749  0.029288     0.102375  0.185224   
                  2006  0.000995  0.006963  0.039294     0.094006  0.195474   
                  2007  0.001860  0.010098  0.036407     0.086102  0.180441   
                  2008  0.000828  0.004140  0.032018     0.085012  0.193486   

                        larceny_theft  m_v_theft  
jurisdiction      year                            
Allegany County   1975       0.611851   0.039931  
                  1976       0.651294   0.034353  
                  1977       0.628675   0.046133  
                  1978       0.652276   0.046926  
                  1979       0.693798   0.042636  
...                               ...        ...  
Washington County 2004       0.586879   0.074115  
                  2005       0.608443   0.068865  
                  2006       0.588660   0.074608  
                  2007       0.621313   0.063779  
                  2008       0.616616   0.067900  

[1000 rows x 7 columns]

## Use Jaccard Similarity by Crime Percent Change to Predict Population

In [34]:
# create df with chosen crime statistics
df_population_rates = df.iloc[:, 8:]
df_population_rates['population'] = df['population']

df_population_rates

grand_total  violent_crime_total  \
jurisdiction      year                                     
Allegany County   1975         2329                  142   
                  1976         2125                   87   
                  1977         2211                  127   
                  1978         2131                  102   
                  1979         2322                  110   
...                             ...                  ...   
Washington County 2004         3643                  536   
                  2005         3790                  521   
                  2006         4021                  568   
                  2007         3763                  506   
                  2008         3623                  442   

                        violent_crime_percent  property_crime_totals  \
jurisdiction      year                                                 
Allegany County   1975                    6.1                   2187   
                  1976                    4.1                   2038   
                  1977                    5.7                   2084   
                  1978                    4.8                   2029   
                  1979                    4.7                   2212   
...                                       ...                    ...   
Washington County 2004                   14.7                   3107   
                  2005                   13.7                   3269   
                  2006                   14.1                   3453   
                  2007                   13.4                   3257   
                  2008                   12.2                   3181   

                        property_crime_percent  \
jurisdiction      year                           
Allegany County   1975                    93.9   
                  1976                    95.9   
                  1977                    94.3   
                  1978                    95.2   
                  1979                    95.3   
...                                        ...   
Washington County 2004                    85.3   
                  2005                    86.3   
                  2006                    85.9   
                  2007                    86.6   
                  2008                    87.8   

                        overall_crime_rate_per_100_000_people  \
jurisdiction      year                                          
Allegany County   1975                                 2923.9   
                  1976                                 2532.1   
                  1977                                 2693.0   
                  1978                                 2664.9   
                  1979                                 2912.7   
...                                                       ...   
Washington County 2004                                 2639.5   
                  2005                                 2693.9   
                  2006                                 2826.0   
                  2007                                 2609.2   
                  2008                                 2485.2   

                        violent_crime_rate_per_100_000_people  \
jurisdiction      year                                          
Allegany County   1975                                  178.3   
                  1976                                  103.7   
                  1977                                  154.7   
                  1978                                  127.6   
                  1979                                  138.0   
...                                                       ...   
Washington County 2004                                  388.4   
                  2005                                  370.3   
                  2006                                  399.2   
                  2007                                  350.9   
                  2008       

In [35]:
# split test/train data
all_ids = df.index.values
training_ids, testing_ids = train_test_split(all_ids, test_size=0.2)

training_ids.shape, testing_ids.shape

((800,), (200,))

In [36]:
k_nn = 10
prediction_rows = []

# find k most similar counties in the training set

for test_id in testing_ids:

    target_obj = set(df_population_rates.loc[test_id[0], test_id[1]].values)
    
    test_county_similarities = []

    for this_id in training_ids:
        this_obj = set(df_population_rates.loc[this_id[0], this_id[1]].values)

        this_intersect = this_obj.intersection(target_obj)
        this_union = this_obj.union(target_obj)

        jaccard = len(this_intersect) / len(this_union)

        test_county_similarities.append({
            "county": this_id,
            "jaccard": jaccard,
        })

    similarity_df = pd.DataFrame(test_county_similarities, columns=["county", "jaccard"])
    top_counties = similarity_df.sort_values(by="jaccard", ascending=False).head(k_nn)

    relevant_populations = []
    for this_id in top_counties["county"]:
        
        # add relevant populations
        relevant_populations.append(df['population'].loc[this_id[0], this_id[1]])
        
    # take average total population
    predicted_populations = np.mean(relevant_populations)

    # track predictions
    prediction_rows.append({
            "county": test_id,
            "population": predicted_populations,
        })
    
predicted_df = pd.DataFrame(prediction_rows).set_index("county")


In [37]:
correct_populations = []

# count number of accurate predictions
for county,row in predicted_df.iterrows():

    actual_population = df['population'].loc[county[0], county[1]]

    # mark correct if prediction is less/more than 40000 of the original total population
    correct_populations.append(1 if row['population'] >= actual_population-40000 and row['population'] <= actual_population+40000 else 0)
    
accuracy = sum(correct_populations) / len(correct_populations)
print("Accuracy:", accuracy)

Accuracy: 0.11


In [38]:
def find_real_population(row):
    return df['population'].loc[row.name[0], row.name[1]]

# create df comparing predicted and real populations
model = predicted_df
model['real_population'] = predicted_df.apply(find_real_population, axis=1)

model

,population,real_population
county,,
"(Dorchester County, 1977)",252232.6,29991
"(Cecil County, 2014)",93373.4,102296
"(Frederick County, 2005)",263072.4,219311
"(Allegany County, 2002)",55548.3,77216
"(Dorchester County, 2006)",156615.3,31487
...,...,...
"(Talbot County, 2013)",101771.3,38253
"(Anne Arundel County, 1977)",436927.8,347538
"(Harford County, 2018)",107116.8,251139


## Find similarities for each County and Year

In [39]:
# create combined features column with all percent change rates
def combined_features(row):
    return str(row['murder_rate_percent_change_per_100_000_people'])+" "+str(row['rape_rate_percent_change_per_100_000_people'])+", "+str(row['robbery_rate_percent_change_per_100_000_people'])+", "+str(row['agg_assault_rate_percent_change_per_100_000_people'])+", "+str(row['b_e_rate_percent_change_per_100_000_people'])+", "+str(row['larceny_theft_rate_percent_change_per_100_000_people'])+", "+str(row['m_v_theft_rate_percent_change_per_100_000_people'])

df['all_change_rates'] = df.apply(combined_features, axis=1)
# count number of features present in each title
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['all_change_rates'])
print("Count Matrix: ", count_matrix.toarray())

# set similarity metric
cosine_sim = cosine_similarity(count_matrix)

Count Matrix:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [40]:
df1 = df.reset_index()
similarity_dict = {}

for index, row in df1.iterrows():
    # generate distances from target county to all other counties
    similar_counties = list(enumerate(cosine_sim[index]))
    
    # sort to get most similar counties first
    sorted_similar_counties = sorted(similar_counties, key = lambda x:x[1], reverse = True)

    similarity_dict[index] = sorted_similar_counties

In [41]:
# sample similarities for target county and year

# Baltimore County, 1982
target_similar_counties = similarity_dict[99]
print(df1['jurisdiction'].loc[99], df1['year'].loc[99])

# print top ten most similar counties
i=0
for county in target_similar_counties:
    print(df1['jurisdiction'].loc[county[0]], df1['year'].loc[county[0]], county[1])
    i = i+1;
    if i>10:
        break

# Montgomery County, 2005
target_similar_counties = similarity_dict[720]
print(df1['jurisdiction'].loc[720], df1['year'].loc[720])

# print top ten most similar counties
i=0
for county in target_similar_counties:
    print(df1['jurisdiction'].loc[county[0]], df1['year'].loc[county[0]], county[1])
    i = i+1;
    if i>10:
        break

# St. Mary's County, 2011
target_similar_counties = similarity_dict[910]
print(df1['jurisdiction'].loc[910], df1['year'].loc[910])

# print top ten most similar countiess
i=0
for county in target_similar_counties:
    print(df1['jurisdiction'].loc[county[0]], df1['year'].loc[county[0]], county[1])
    i = i+1;
    if i>10:
        break

Baltimore City 1982
Baltimore City 1982 1.0000000000000002
Baltimore County 1999 0.8728715609439696
Baltimore County 1982 0.8660254037844388
Baltimore City 1977 0.816496580927726
Baltimore County 1996 0.816496580927726
Montgomery County 1990 0.816496580927726
Charles County 1996 0.7071067811865477
Anne Arundel County 1987 0.6666666666666669
Anne Arundel County 2001 0.6666666666666669
Anne Arundel County 2019 0.6666666666666669
Baltimore City 1980 0.6666666666666669
